In [2]:
from sklearn import datasets                                
import pandas as pd
import numpy as np
import math

In [3]:
iris = datasets.load_iris()                                

In [4]:
df = pd.DataFrame(iris.data)                            
df.columns = ["sl", "sw", 'pl', 'pw']

In [5]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [6]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
#df

In [7]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [8]:
y1 = pd.DataFrame(iris.target)                                  
unused_features = list(df.columns)     #list containing all feature names                        
y=y1.values
x=df.values

In [9]:
def entropy(s,c):                      #funtion to calculate the entropy of a particular node                                          
    ent=0
    for k in c:
        if s!=0:
            n=k/s
            if n!=0:
                ent+=(-n*math.log(n,10))
    return ent

In [10]:
def avg_all(ts, s, e):                 #funtion to calculate the info gain and split info of a feature                
    info_f=0
    split_info=0
    for k in range(len(e)):
        if s!=0:
            info_f+=ts[k]/s*e[k]
            split_info+=-(ts[k]/s*math.log((ts[k]/s),10))
    return info_f,split_info

In [11]:
def gain(df,f,y):
    if f=="sl_labeled":                      #values of the feature are placed in a list
        sf=list(df["sl_labeled"])
    elif f=="sw_labeled":
        sf=list(df["sw_labeled"])
    elif f=="pl_labeled":
        sf=list(df["pl_labeled"])
    else:
        sf=list(df["pw_labeled"])
        
    ts=[]
    entro=[]
    for i in set(sf):
        c0=0
        c1=0
        c2=0
        for j in range(len(y)):
            if sf[j]==i:
                if y[j]==0:
                    c0+=1
                elif y[j]==1:
                    c1+=1
                elif y[j]==2:
                    c2+=1
        sum_ = c0+c1+c2
        ts.append(sum_)                     #ts is a list which contains the sizes of nodes of the current feature(passed to the function)
        c=[c0,c1,c2]
        en = entropy(sum_,c)                #passing the sum and the counts of classes(0,1,2) to entropy
        entro.append(en)                    #list appending entropies of all nodes(a,b,c,d) of of the current feature
    a,b=avg_all(ts, sum(ts), entro)         
    return a,b                              #returing the info gain and the split info of the current feature

In [12]:
def next_split_f(e, df, y, unused_features):        #function which returns the feature with the max gain ratio for next split
    gain_max=-1
    gain_max=float(gain_max)
    selected=""
    if len(unused_features)!=0:
        for f in unused_features:                   #iterating over all unused features and finding gain ratio of each
            i_f,s_f= gain(df,f,y)                   #i_f is the info gain and s_f is the split info of the feature
            info= e-i_f                             #e is the entropy of node which is splitted
            gain_ratio= info/s_f
            if gain_max <= gain_ratio:
                gain_max= gain_ratio
                selected=f  
        return selected,gain_max                   #returning the feature with its gain ratio upon which the next split has to be done

In [13]:
def all_(sg,abcd,df,y,feat):                       #funtion for calculating and printing everything we require (above level 0)      
    while unused_features!=[]:
        list_=list(df[feat])                       #list of the selected feature values      
        for i in abcd:                                   
            print("Level",sg)
            c0=0
            c1=0
            c2=0
            for j in range(len(y)):                #calculating the count of each class                   
                if list_[j]==i:
                    if y[j]==0:
                        c0+=1
                    elif y[j]==1:
                        c1+=1
                    else:
                        c2+=1
            c=[c1,c2,c0]
            e=entropy(c1+c2+c0,c)                  #calculating entropy                     
            print("Count of 0 =",c0)                    
            print("Count of 1 =",c1)
            print("Count of 2 =",c2)  
            print("Current Entropy  is =",e)
            if e!=0:                               #if the entropy is not equal to 0, the node isn't pure, further splitting is required (this node will act as root for the split)   
                y3=[]                              #y3, df1 stores the new target and new data (for the next split)
                df1=df[df[feat]==i]
                for p in range(len(list_)):
                    if list_[p]==i:
                        y3.append(y[p])
                if len(unused_features)!=0:        
                    unused_features.remove(feat)   #remove the feature which is used for this split
                if len(unused_features)==0:
                    print("No features left to split")  #if no feature is left for further splitting, 'no features left' is printed
                else:                              #if features are left, find the feature with maximum gain ratio for the next split
                    feat,ig=next_split_f(e,df1, y3, unused_features)   #it gives the feature with maximum gain ratio
                    print("Splitting on feature","'"+feat+"'","with gain ratio",ig)          #feature selected
            else:                                  #if entropy is 0 we reached a pure node, no splitting is now required
                print("Reached leaf Node")
            print()
        df=df1
        y=y3
        abcd=set(df[feat])                         #set which contain the unique values in the selected feature
        abcd=list(abcd)
        abcd.sort()
        sg=sg+1
        all_(sg,abcd,df,y,feat)                    #calling funtion all_ recursively till all features are used for splitting

In [14]:
c0=len(y[y==0])                                      #calculating the count of each class for level 0          
c1=len(y[y==1])
c2=len(y[y==2])
c=[c1,c2,c0]                                                       
e=entropy(c1+c2+c0,c)                                #calculating entropy
print("Level",0)                                               
print("Count of 0 =",c0)
print("Count of 1 =",c1)
print("Count of 2 =",c2)
print("Current Entropy is =",e)
feat,ig=next_split_f(e, df, y, unused_features)      #calling next_split_feat which gives us the feature with the maximum gain ratio for the next split
print("Splitting on feature","'"+feat+"'","with gain ratio",ig)          #feature selected
print()
abcd=set(df[feat])                                   #set which contain the unique values in the selected feature          
abcd=list(abcd)
abcd.sort()                                                     
sg=1                                                 #count for the levels       
all_(sg,abcd,df,y,feat)  

Level 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 0.4771212547196624
Splitting on feature 'pw_labeled' with gain ratio 0.6996382036222092

Level 1
Count of 0 = 50
Count of 1 = 0
Count of 2 = 0
Current Entropy  is = 0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 10
Count of 2 = 0
Current Entropy  is = 0.0
Reached leaf Node

Level 1
Count of 0 = 0
Count of 1 = 40
Count of 2 = 16
Current Entropy  is = 0.25982518101310587
Splitting on feature 'pl_labeled' with gain ratio 0.43340994956210654

Level 1
Count of 0 = 0
Count of 1 = 0
Count of 2 = 34
Current Entropy  is = 0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 1
Count of 2 = 0
Current Entropy  is = 0.0
Reached leaf Node

Level 2
Count of 0 = 0
Count of 1 = 39
Count of 2 = 8
Current Entropy  is = 0.19813531389382344
Splitting on feature 'sl_labeled' with gain ratio 0.12674503775809323

Level 2
Count of 0 = 0
Count of 1 = 0
Count of 2 = 8
Current Entropy  is = 0.0
Reached leaf Node

Level 

# Implementation of Tree

In [25]:
from sklearn.tree import export_graphviz

In [26]:
from sklearn.tree import DecisionTreeClassifier as tree
data = datasets.load_iris()                                
clf=tree()
clf.fit(data.data,data.target)
import pydotplus
dotdata=export_graphviz(clf,out_file=None,
                       feature_names=data.feature_names,class_names=data.target_names)
graph=pydotplus.graph_from_dot_data(dotdata)
graph.write_pdf('Decision_Tree.pdf')

True